In [1]:
%load_ext autoreload
%autoreload 2

## В этом ноутбуке мы обучим и сохраним модель классификации породы собак на изображении. Будем файн-тьюнить Resnet50. Обучим модель на датасете с породами собак https://www.kaggle.com/competitions/dog-breed-identification/overview, а применить попробуем к нашему дата сету с кошками и собаками. 

In [2]:
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [3]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
from torchvision import models
import torchvision.transforms as transforms
import torch.optim as optim
from skimage import io
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# mapping labels same order of sample_submittion.csv
map_labels = dict()
for index, value in enumerate(pd.read_csv('../data/dog-breed-identification/sample_submission.csv').columns[1:]):
    map_labels[value] = index

In [5]:
# load custom dataset
class DogBreedsDataset_train(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform['train']
        
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.labels.iloc[index, 0])+'.jpg'
        image = io.imread(img_path)
        y_label = torch.tensor(int(map_labels[self.labels.iloc[index, 1]]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [6]:
# load custom dataset
class DogBreedsDataset_test(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.name_images = pd.read_csv(csv_file)['id'].values
        self.root_dir = root_dir
        self.transform = transform['test']
    
        
    def __len__(self):
        return len(self.name_images)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.name_images[index])+'.jpg'
        image = io.imread(img_path)

        if self.transform:
            image = self.transform(image)

        return image

In [7]:
# Hyperparameters
in_channel = 3
num_classes = 120
learning_rate = 5*1e-4
batch_size = 128
num_epochs = 30

In [8]:
# images preprocessing
transform ={'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(size=256, scale=(0.95, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])]),  # Imagenet standards
    'test': transforms.Compose([transforms.ToPILImage(),
                                      transforms.RandomResizedCrop(size=256, scale=(0.95, 1.0)),
                                      transforms.CenterCrop(size=224),
                                      transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])}

dataset = DogBreedsDataset_train('../data/dog-breed-identification/labels.csv', '../data/dog-breed-identification/train', transform)
# Random split data to 70% training and 30% validation
train_set_size = int(len(dataset) * 0.7)
valid_set_size = len(dataset) - train_set_size
train_set, valid_set = torch.utils.data.random_split(dataset, [train_set_size, valid_set_size])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True)

# loading testing data
test_data = DogBreedsDataset_test('../data/dog-breed-identification/sample_submission.csv', '../data/dog-breed-identification/test', transform=transform)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [9]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# load model architecture with weights (resnet50) 
model = models.resnet50(pretrained=True)

# freeze all layers 
for param in model.parameters():
    param.requires_grad = False
    
# replace last layer with from 1000 classes to be 120 classes
model.fc = nn.Linear(2048, 120)
model = model.to(device=device)
model

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# checkpoint to save and load model
checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}

In [12]:
def traindata(device, model, epochs, optimizer, loss_function, train_loader, valid_loader):
    # Early stopping
    best_loss = 100
    patience = 5
    triggertimes = 0
    
    for epoch in range(1, epochs+1):
        model.train()
        correct = 0
        loss_total = 0
        for data in train_loader:
            input = data[0].to(device)
            label = data[1].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward and backward propagation
            output = model(input)
            _, predicted = output.max(1)
            loss = loss_function(output, label)
            loss_total += loss.item()
            correct+= (predicted == label).sum()
            loss.backward()
            optimizer.step()
        print('epoch number: {}'.format(epoch))
        print('Training Accuracy: {} Training loss: {}'.format(correct/len(train_loader.sampler), loss_total/len(train_loader)))
        
        # Early stopping
        current_loss = validation(model, device, valid_loader, loss_function)
        print('The Current Loss:', current_loss)
        print('Best Loss:', best_loss)

        if current_loss > best_loss:
            trigger_times += 1
            print('Trigger Times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model

        else:
            print('trigger times: 0')
            trigger_times = 0
            #save_checkpoint(checkpoint)
            best_loss = current_loss

    return model

In [14]:
def validation(model, device, valid_loader, loss_function):

    model.eval()
    loss_total = 0
    correct = 0
    # Test validation data
    with torch.no_grad():
        for data in valid_loader:
            input = data[0].to(device)
            label = data[1].to(device)

            output = model(input)
            _, predicted = output.max(1)
            loss = loss_function(output, label)
            loss_total += loss.item()
            correct+= (predicted == label).sum()
        print('Validation Accuracy: {}'.format(correct/len(valid_loader.sampler)))
    return loss_total / len(valid_loader)

In [15]:
%%time
model = traindata(device, model, num_epochs, optimizer, criterion, train_loader, valid_loader)

epoch number: 1
Training Accuracy: 0.75052410364151 Training loss: 1.6510114989110403
Validation Accuracy: 0.7574176788330078
The Current Loss: 1.3101984361807506
Best Loss: 100
trigger times: 0
epoch number: 2
Training Accuracy: 0.8132774233818054 Training loss: 1.0687778634684426
Validation Accuracy: 0.7896968126296997
The Current Loss: 0.9936371048291525
Best Loss: 1.3101984361807506
trigger times: 0
epoch number: 3
Training Accuracy: 0.8401117920875549 Training loss: 0.8409305544836181
Validation Accuracy: 0.7932833433151245
The Current Loss: 0.8937988728284836
Best Loss: 0.9936371048291525
trigger times: 0
epoch number: 4
Training Accuracy: 0.8554856777191162 Training loss: 0.7112004033156804
Validation Accuracy: 0.8095859289169312
The Current Loss: 0.7838404253125191
Best Loss: 0.8937988728284836
trigger times: 0
epoch number: 5
Training Accuracy: 0.8679245114326477 Training loss: 0.621671157756022
Validation Accuracy: 0.8004564642906189
The Current Loss: 0.7607966214418411
Best 

## Сохраним модель и вызовем в 3_evaluation 

In [16]:
model_path = "../data/dogs_breed"
torch.save(model.state_dict(), model_path)